# Walmart : predict weekly sales 🚧

Project summary :

#### EDA and preprocessing
 - EDA
 - PREPROCESSING
#### Linear regression model (baseline)
 - Coefficients
#### Limit overfitting by training a regularized regression model
 - CROSS VALIDATED SCORE 
 - RIDGE
 - LASSO
 - EVALUATION

In [133]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

In [134]:
# import & display the first 5 rows of the dataset
df = pd.read_csv('Walmart_Store_sales.csv')
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [135]:
# display the number of rows
print("Number of rows : {}".format(df.shape[0]))
print()
# show the percentage of missing values
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])
# show the description of the data in the DataFrame
df.describe(include="all")

Number of rows : 150

Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000


In [136]:
# Display Weekly sales (target) distribution
fig = px.box(df, y="Weekly_Sales",
                   title= "Weekly sales distribution"
                   )

fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  yaxis_title = 'Weely sales',
                  xaxis = {'visible': False},
                  template = 'plotly_dark'
                  )

fig.show()

In [137]:
# Display weekly sales by stores
fig = px.histogram(df, x = "Store",
                   y="Weekly_Sales",
                   title = 'Weekly sales by stores',
                   text_auto=True
                  )
fig.update_traces(textposition = 'outside', textfont_size = 15)                 
fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  xaxis_title = 'Stores',
                  yaxis_title = 'Weekly sales',
                  template = 'plotly_dark'
                  )                  
fig.show()

In [138]:
# import datetime to manage dates
import datetime

print(type(df.Date[0]))
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True)
print(type(df.Date[0]))
df['Date'] = df['Date'].dt.strftime('%Y/%m/%d')
df = df.sort_values(by='Date')

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [139]:
# Manage dates creating new columns about Year,	Month Day, DayOfWeek & DayName
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True)
df['Year'] = df['Date'].dt.strftime('%Y')
df['Month'] = df['Date'].dt.strftime('%m')
df['Day'] = df['Date'].dt.strftime('%d')
df['DayOfWeek'] = df['Date'].dt.day_of_week
df['DayName'] = df['Date'].dt.day_name()
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek,DayName
67,3.0,2010-02-05,461622.22,0.0,45.71,2.572,214.424881,7.368,2010,02,05,4.0,Friday
44,1.0,2010-02-12,1641957.44,1.0,38.51,2.548,211.242170,8.106,2010,02,12,4.0,Friday
107,8.0,2010-02-12,994801.40,1.0,33.34,2.548,214.621419,6.299,2010,02,12,4.0,Friday
112,2.0,2010-02-19,2124451.54,0.0,39.69,2.514,210.945160,NaN,2010,02,19,4.0,Friday
115,15.0,2010-02-19,660838.75,0.0,NaN,2.909,131.637000,NaN,2010,02,19,4.0,Friday


In [140]:
# display the first 5 rows of the new dataset  
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek,DayName
67,3.0,2010-02-05,461622.22,0.0,45.71,2.572,214.424881,7.368,2010,02,05,4.0,Friday
44,1.0,2010-02-12,1641957.44,1.0,38.51,2.548,211.242170,8.106,2010,02,12,4.0,Friday
107,8.0,2010-02-12,994801.40,1.0,33.34,2.548,214.621419,6.299,2010,02,12,4.0,Friday
112,2.0,2010-02-19,2124451.54,0.0,39.69,2.514,210.945160,NaN,2010,02,19,4.0,Friday
115,15.0,2010-02-19,660838.75,0.0,NaN,2.909,131.637000,NaN,2010,02,19,4.0,Friday


In [141]:
# check outliers
fig = px.box(df, y="Temperature",
                   title= 'Box Plot: Temperature',
                   )

fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  yaxis_title = 'Temperature',
                  xaxis = {'visible': False},
                  template = 'plotly_dark'
                  )

fig.show()

In [142]:
# check outliers
fig = px.box(df, y="Fuel_Price",
                   title= 'Box Plot: Fuel_Price',
                   )

fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  yaxis_title = 'Fuel_Price',
                  xaxis = {'visible': False},
                  template = 'plotly_dark'
                  )

fig.show()

In [143]:
# check outliers
fig = px.box(df, y="CPI",
                   title= 'Box Plot: CPI',
                   )

fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  yaxis_title = 'CPI',
                  xaxis = {'visible': False},
                  template = 'plotly_dark'
                  )

fig.show()

In [144]:
# check outliers
fig = px.box(df, y="Unemployment",
                   title= 'Box Plot: Unemployment',
                   )

fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  yaxis_title = 'Unemployment',
                  xaxis = {'visible': False},
                  template = 'plotly_dark'
                  )

fig.show()

#### Only Unemployment column contain outliers

In [145]:
# remove outliers
print("Shape of dataset before dropping outliers :", df.shape)

#calculate upper and lower limits
upper_limit = df.Unemployment.mean() + 3 * df.Unemployment.std()
lower_limit = df.Unemployment.mean() - 3 * df.Unemployment.std()

#outliers removed keeping missing values
df = df[(df.Unemployment < upper_limit) & (df.Unemployment > lower_limit) | (df.Unemployment.isnull())]

print("Shape of dataset after dropping outliers :", df.shape)

Shape of dataset before dropping outliers : (150, 13)
Shape of dataset after dropping outliers : (145, 13)


In [146]:
# remove row with missing values in the target column & reset the indices
df = df.dropna(subset=['Weekly_Sales'])
df = df.reset_index(drop=True)

In [147]:
# show the percentage of missing values in Weekly Sales column
print("Percentage of missing values: ")
display(100*df['Weekly_Sales'].isnull().sum()/df['Weekly_Sales'].shape[0])

Percentage of missing values: 


0.0

In [148]:
# Drop the column Date
df = df.drop(['Date'], axis= 1)
df.columns

Index(['Store', 'Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price',
       'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'DayOfWeek', 'DayName'],
      dtype='object')

# Preprocessing

In [149]:
# Separe features from target
features_list = ['Store', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month']
target_variable = 'Weekly_Sales'

X = df.loc[:, features_list]
Y = df.loc[:, target_variable]

# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [150]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer

# Create pipeline for numeric features
numeric_features = ['Temperature',
       'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month'] # Names of numeric columns in X_train/X_test

numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer()), # missing values are imputed using the mean value from n_neighbors
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = ['Store', 'Holiday_Flag'] # Names of categorical columns in X_train/X_test

categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [151]:
# Preprocessings on traint set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) 
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit again !! 
print('...Done.')
print(X_test[0:5,:])

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
118   10.0           NaN        43.43       3.287  127.191774         8.744   
68     5.0           0.0        89.42       3.682  216.046436         6.529   
16     3.0           0.0        78.53       2.705  214.495838         7.343   
96    19.0           0.0        55.20       4.170  137.923067         8.150   
122   17.0           0.0          NaN       3.793  131.037548         6.235   

     Year Month  
118   NaN   NaN  
68   2011    07  
16   2010    06  
96   2012    04  
122   NaN   NaN  
...Done.
  (0, 0)	-0.9431128158310235
  (0, 1)	-0.04122721657157029
  (0, 2)	-1.3776857605677293
  (0, 3)	1.3917869284207915
  (0, 4)	-0.5557868568380021
  (0, 5)	-0.030500035869273634
  (0, 14)	1.0
  (1, 0)	1.660585496078656
  (1, 1)	0.7956213142912069
  (1, 2)	0.9616323193914564
  (1, 3)	-0.8777243085014366
  (1, 4)	0.20476357883496238
  (1, 5)	0.17636977263536763
  (1, 

# Linear Regression

In [152]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [153]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred[0:5])

Predictions on training set...
...Done.
[1926830.74412334  164525.19760676  363586.49283353 1303022.44778058
  834603.06447234]


In [154]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred[0:5])

Predictions on test set...
...Done.
[ 501295.16199745 1376369.84666004 1895365.79354539  684417.28586788
  781207.31646741]


In [155]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9694870784189158
R2 score on test set :  0.9420734257705164


## COEFFICIENTS

In [156]:
# Take features names from the preprocessing
feat_1 = preprocessor.transformers_[1][1].steps[1][1].get_feature_names().tolist()
feat_2 = X.columns[:-2].to_list()

# Transform in array the two list and concatenate
features = np.array(feat_1 + feat_2)

# Create datafreame from features names and model coefficients
coefs = pd.DataFrame([regressor.coef_],columns=features).T.reset_index()

In [157]:
# Rename columns
coefs.columns = ['features','Coefficients']

# Change coefficients values in absolute values 
coefs.Coefficients = coefs.Coefficients.abs()

In [158]:
# display feature coefficients for the baseline model
fig = px.bar(coefs.sort_values(by=['Coefficients'], ascending=True), y= 'features', x = "Coefficients",
                   title = 'Features coefficients for the baseline model',
                   text_auto=True,
                   height= 500
                  )
fig.update_traces(textposition = 'outside', textfont_size = 15)                 
fig.update_layout(title_x = 0.5,
                  margin=dict(l=50,r=50,b=50,t=50,pad=4),
                  yaxis_title = 'Features',
                  xaxis = {'visible': False}, 
                  template = 'plotly_dark'
                  )                  
fig.show()

##### Top 5 coefs are all of them from Store

## CROSS VALIDATED SCORE

In [159]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

# Perform 5-fold cross-validation to evaluate the generalized R2 score obtained with our model
print("3-fold cross-validation...")
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.8430575692555873
The standard deviation is :  0.12577281447213212


## Cross-validated score for a Ridge model (with default value of λ)

In [160]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
print("3-fold cross-validation...")
regressor = Ridge()
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.7594109317858566
The standard deviation is :  0.22431960498416723


In [161]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.07, 0.075, 0.08, 0.085] # 0 corresponds to no regularization
}
gridsearch_ridge = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
gridsearch_ridge.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_ridge.best_params_)
print("Best R2 score : ", gridsearch_ridge.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.075}
Best R2 score :  0.8489963525840984


In [162]:
# Create variable containing the target prediction on train set
Y_train_pred_ridge = gridsearch_ridge.predict(X_train)
# Create variable containing the target prediction on test set
Y_test_pred_ridge = gridsearch_ridge.predict(X_test)

# Cross-validated score for a Lasso model

In [163]:
from sklearn.linear_model import Lasso
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Lasso model
print("3-fold cross-validation...")
regressor = Lasso()
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.8430450021675181
The standard deviation is :  0.12582507983924301


In [164]:
# Perform grid search
print("Grid search...")
regressor = Lasso()
# Grid of values to be tested
params = {
    'alpha': [0.000001, 0.00001, 0.0001, 0.0000001] # 0 corresponds to no regularization
}
gridsearch_lasso = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
gridsearch_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R2 score : ", gridsearch_lasso.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 1e-07}
Best R2 score :  0.8430574780144836


In [165]:
# Create variable containing the target prediction on train set
Y_train_pred_lasso = gridsearch_lasso.predict(X_train)
# Create variable containing the target prediction on test set
Y_test_pred_lasso = gridsearch_lasso.predict(X_test)

## EVALUATION 

In [166]:
# Make predictions & Display R2 scores for this model
print("R2 score on training set: ", r2_score(Y_train, Y_train_pred_ridge))
print("R2 score on test set: ", r2_score(Y_test, Y_test_pred_ridge))

R2 score on training set:  0.9687646073986017
R2 score on test set:  0.9388080829345219


In [167]:
# # Make predictions & Display R2 scores for this model
print("R2 score on training set: ", r2_score(Y_train, Y_train_pred_lasso))
print("R2 score on test set: ", r2_score(Y_test, Y_test_pred_lasso))

R2 score on training set:  0.9694870784207381
R2 score on test set:  0.942073298410748


##### All the models have a very good R2 Score

##### But Lasso overfitting is lower then the Rigde model